## Scrapping

This notebook is used to scrap [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M] and obtain the list of postal codes of canada

In [1]:
import pandas as pd

from urllib import request
from bs4 import BeautifulSoup

Load webpage and obtain html

In [2]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = request.urlopen(html).read()

soup = BeautifulSoup(source, "html.parser")
table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')

get table elements from html

In [3]:
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

In [4]:
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


remove cases where borough is not assigned

In [5]:
df = df[df.Borough != "Not assigned"]

join various neighbourhoods in the same borough

In [6]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

replcae not assigned neighbourhoods with borough name

In [7]:
def replace(r):
    if r['Neighborhood'] == "Not assigned": r["Neighborhood"] = r["Borough"]
    return r

In [8]:
df = df.apply(replace,axis=1)

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

## Geocode

The geocoder api did not work, so i downloaded the csv and used

```python
import geocoder # import geocoder

lat = []
long = []

for _, r in df.iterrows():

    postal_code = r['PostalCode']
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]  
```

load coordinatess and merge with the data

In [11]:
lat_long = pd.read_csv("Geospatial_Coordinates.csv")

In [12]:
df = df.merge(lat_long, left_on ="PostalCode", right_on="Postal Code").drop(columns=["Postal Code"])

In [13]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Map

In [14]:
import folium
from geopy.geocoders import Nominatim

plot thr map of toronto

In [15]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

select s borough to work with

In [16]:
BOROUGH = "Scarborough"

In [17]:
borough_data = df[df['Borough'] == BOROUGH].reset_index(drop=True)
borough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


get the coorinates

In [18]:
address = BOROUGH + ', Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(BOROUGH, latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


map the borough

In [19]:
# create map of Manhattan using latitude and longitude values
map_borough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(borough_data['Latitude'], borough_data['Longitude'], borough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough)  
    
map_borough

### Explore the location

In [20]:
import json
import requests

credentials = json.load(open('foursquare.json'))

CLIENT_ID = credentials['clientID'] # your Foursquare ID
CLIENT_SECRET = credentials['clientSecret'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

get nearby venues for each neighborhood in the borough

In [22]:
borough_venues = getNearbyVenues(names=borough_data['Neighborhood'],
                                   latitudes=borough_data['Latitude'],
                                   longitudes=borough_data['Longitude'])


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [23]:
print(borough_venues.shape)
borough_venues.head()

(91, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


get number of venues in each neighbourhood

In [24]:
borough_venues.groupby('Neighborhood').count().max(axis=1)

Neighborhood
Agincourt                                                    5
Agincourt North, L'Amoreaux East, Milliken, Steeles East     3
Birch Cliff, Cliffside West                                  4
Cedarbrae                                                    7
Clairlea, Golden Mile, Oakridge                             10
Clarks Corners, Sullivan, Tam O'Shanter                      9
Cliffcrest, Cliffside, Scarborough Village West              3
Dorset Park, Scarborough Town Centre, Wexford Heights        9
East Birchmount Park, Ionview, Kennedy Park                  7
Guildwood, Morningside, West Hill                            8
Highland Creek, Rouge Hill, Port Union                       1
L'Amoreaux West, Steeles West                               13
Maryvale, Wexford                                            6
Rouge, Malvern                                               1
Scarborough Village                                          2
Woburn                                    

In [25]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 53 uniques categories.


In [26]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Neighborhood'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [27]:
borough_onehot.shape

(91, 54)

mean requency occurance

In [28]:
borough_grouped = borough_onehot.groupby('Neighborhood').mean().reset_index()
borough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.000000,...,0.000000,0.000,0.200000,0.000000,0.20,0.0,0.000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.333333,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000,0.000000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.000000,...,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.142857,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.100000,...,0.000000,0.000,0.000000,0.000000,0.00,0.1,0.000,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.111111,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.111111,0.111111
8,"East Birchmount Park, Ionview, Kennedy Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.142857,...,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
9,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.0,0.000000,...,0.000000,0.125,0.000000,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.000000


top 5 venues in each location

In [29]:
num_top_venues = 5

for hood in borough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park  0.67
1                 Playground  0.33
2        American Restaurant  0.00
3          Korean Restaurant  0.00
4  Latin American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2          Park  

get top 10 venues for each location

In [30]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = borough_grouped['Neighborhood']

for ind in np.arange(borough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,College Stadium,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Chinese Restaurant,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Convenience Store


## Clustering

create clusters, and then merge clusters with neighborhoods and most common venues

In [32]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

borough_grouped_clustering = borough_grouped.drop('Neighborhood', 1)

Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(borough_grouped_clustering)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()


NameError: name 'kmeans' is not defined

In [33]:
# set number of clusters
kclusters = 7

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_grouped_clustering)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

borough_merged = borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
borough_merged = borough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

borough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1.0,Bar,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Spa,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,General Entertainment


visualiz on map

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(borough_merged['Latitude'], borough_merged['Longitude'], borough_merged['Neighborhood'], borough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    except Exception as e:
        pass
map_clusters

examine the clusters

In [35]:
borough_merged.loc[borough_merged['Cluster Labels'] == 0, borough_merged.columns[[1] + list(range(5, borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Spa,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium
4,Scarborough,0.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,General Entertainment
6,Scarborough,0.0,Coffee Shop,Convenience Store,Bus Station,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gaming Cafe
7,Scarborough,0.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Convenience Store
9,Scarborough,0.0,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,0.0,Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Gaming Cafe,Thrift / Vintage Store,Latin American Restaurant,Vietnamese Restaurant,Rental Car Location,Soccer Field
11,Scarborough,0.0,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Gaming Cafe,Furniture / Home Store
12,Scarborough,0.0,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,College Stadium,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint
13,Scarborough,0.0,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,Gaming Cafe,Furniture / Home Store
15,Scarborough,0.0,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Electronics Store,Discount Store,Pharmacy,Pizza Place,Breakfast Spot,Sandwich Place,Coffee Shop


In [36]:
borough_merged.loc[borough_merged['Cluster Labels'] == 1, borough_merged.columns[[1] + list(range(5, borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [37]:
borough_merged.loc[borough_merged['Cluster Labels'] == 2, borough_merged.columns[[1] + list(range(5, borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store


In [38]:
borough_merged.loc[borough_merged['Cluster Labels'] == 3, borough_merged.columns[[1] + list(range(5, borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,Coffee Shop,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [39]:
borough_merged.loc[borough_merged['Cluster Labels'] == 4, borough_merged.columns[[1] + list(range(5, borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,Movie Theater,College Stadium,General Entertainment,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
